# Credit Risk Data Cleaning and Feature Engineering

This notebook performs data wrangling and feature engineering on a credit risk dataset sourced from Kaggle. The dataset includes borrower-level financial data, with a target variable indicating whether a serious delinquency occurred within two years. The cleaned data will later be used for SQL-based exploratory data analysis (EDA).

Goals:
- Remove invalid or extreme values
- Create new features like total delinquency counts and income/age buckets
- Prepare a structured dataset ready for SQL analysis


## Importing Dependencies and csv file

In [18]:
# Dependencies
import pandas as pd
import numpy as np

df=pd.read_csv('../data/Credit Risk Benchmark Dataset.csv', encoding_errors='ignore')
df.head()

,rev_util,age,late_30_59,debt_ratio,monthly_inc,open_credit,late_90,real_estate,late_60_89,dependents,dlq_2yrs
0,0.006999,38.0,0.0,0.302150,5440.0,4.0,0.0,1.0,0.0,3.0,0
1,0.704592,63.0,0.0,0.471441,8000.0,9.0,0.0,1.0,0.0,0.0,0
2,0.063113,57.0,0.0,0.068586,5000.0,17.0,0.0,0.0,0.0,0.0,0
3,0.368397,68.0,0.0,0.296273,6250.0,16.0,0.0,2.0,0.0,0.0,0
4,1.000000,34.0,1.0,0.000000,3500.0,0.0,0.0,0.0,0.0,1.0,0


## Data Cleaning

Checking to see if there are any null values present.

In [19]:
df.isnull().sum()

rev_util       0
age            0
late_30_59     0
debt_ratio     0
monthly_inc    0
open_credit    0
late_90        0
real_estate    0
late_60_89     0
dependents     0
dlq_2yrs       0
dtype: int64

Checking column statistics for data cleaning.

In [20]:
df.describe()

,rev_util,age,late_30_59,debt_ratio,monthly_inc,open_credit,late_90,real_estate,late_60_89,dependents,dlq_2yrs
count,16714.000000,16714.000000,16714.000000,16714.000000,16714.000000,16714.000000,16714.000000,16714.000000,16714.000000,16714.000000,16714.000000
mean,4.799862,48.798672,1.110267,30.980298,6118.120258,8.503709,0.863827,1.047445,0.734354,0.944358,0.500000
std,204.062345,13.906078,7.172890,719.694859,5931.841779,5.370965,7.167576,1.272565,7.138737,1.198791,0.500015
min,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.082397,38.000000,0.000000,0.155971,3128.500000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.443080,48.000000,0.000000,0.322299,5000.000000,8.000000,0.000000,1.000000,0.000000,0.000000,0.500000
75%,0.926637,58.000000,1.000000,0.533426,7573.000000,11.000000,0.000000,2.000000,0.000000,2.000000,1.000000
max,22000.000000,101.000000,98.000000,61106.500000,250000.000000,57.000000,98.000000,29.000000,98.000000,8.000000,1.000000


### Data Cleaning Summary

Based on initial inspection via `df.describe()`, several features contained extreme outliers:

- `rev_util` had values over 22,000 (22000% utilization)
- `debt_ratio` had values over 61,000
- `monthly_inc` had incomes up to $250,000/month
- `late_*` variables had counts up to 98 delinquencies

To clean the data:
- Rows with implausible values in the above features will be **removed**
- Rows with income = 0 will be **dropped**
- `rev_util` will be **capped** to 1.5 (150%) to handle over-limit behavior

The resulting dataset will be free of unrealistic outliers and is better suited for meaningful analysis.


In [21]:
df = df[
    (df['rev_util'] <= 1.5) &
    (df['debt_ratio'] <= 10) &
    (df['monthly_inc'] <= 50000) &
    (df['monthly_inc'] > 0) &
    (df['late_30_59'] <= 10) &
    (df['late_60_89'] <= 10) &
    (df['late_90'] <= 10)
]

df.describe()

,rev_util,age,late_30_59,debt_ratio,monthly_inc,open_credit,late_90,real_estate,late_60_89,dependents,dlq_2yrs
count,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000
mean,0.488989,48.917934,0.588123,0.413656,6095.084353,8.578544,0.330614,1.055988,0.209739,0.943332,0.496539
std,0.397583,13.878219,1.114313,0.484401,4452.196679,5.345083,0.912839,1.253517,0.604156,1.197948,0.500003
min,0.000000,21.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.080565,39.000000,0.000000,0.157773,3244.500000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.432673,48.000000,0.000000,0.320105,5000.000000,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.914030,58.000000,1.000000,0.523145,7650.000000,11.000000,0.000000,2.000000,0.000000,2.000000,1.000000
max,1.491694,101.000000,10.000000,9.184035,50000.000000,57.000000,10.000000,25.000000,8.000000,8.000000,1.000000


In [22]:

# Create count and binary delinquency indicators
df['total_late'] = df['late_30_59'] + df['late_60_89'] + df['late_90']
df['any_late'] = (df['total_late'] > 0).astype(int)

df.describe()

,rev_util,age,late_30_59,debt_ratio,monthly_inc,open_credit,late_90,real_estate,late_60_89,dependents,dlq_2yrs,total_late,any_late
count,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000,16182.000000
mean,0.488989,48.917934,0.588123,0.413656,6095.084353,8.578544,0.330614,1.055988,0.209739,0.943332,0.496539,1.128476,0.417007
std,0.397583,13.878219,1.114313,0.484401,4452.196679,5.345083,0.912839,1.253517,0.604156,1.197948,0.500003,1.926078,0.493079
min,0.000000,21.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.080565,39.000000,0.000000,0.157773,3244.500000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.432673,48.000000,0.000000,0.320105,5000.000000,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.914030,58.000000,1.000000,0.523145,7650.000000,11.000000,0.000000,2.000000,0.000000,2.000000,1.000000,2.000000,1.000000
max,1.491694,101.000000,10.000000,9.184035,50000.000000,57.000000,10.000000,25.000000,8.000000,8.000000,1.000000,17.000000,1.000000


## Feature Engineering

In [25]:
# Age groups
df['age_group'] = pd.cut(df['age'], bins=[0, 30, 45, 60, 120], labels=['<30', '30–45', '45–60', '60+'])

# Income brackets
df['income_bracket'] = pd.cut(df['monthly_inc'], bins=[0, 2500, 5000, 10000, np.inf], labels=['<2.5k', '2.5k–5k', '5k–10k', '10k+'])

# Debt ratio risk categories
df['debt_bucket'] = pd.cut(df['debt_ratio'], bins=[0, 0.3, 0.6, 1, np.inf], labels=['Low', 'Medium', 'High', 'Extreme'])

# Utilization bands
df['util_band'] = pd.cut(df['rev_util'], bins=[0, 0.3, 0.6, 0.9, 1, np.inf], labels=['Low', 'Medium', 'High', 'Maxed', 'Overlimit'])

print(df['age_group'].value_counts())
print(df['income_bracket'].value_counts())
print(df['debt_bucket'].value_counts())
print(df['util_band'].value_counts())



age_group
45–60    5944
30–45    5348
60+      3377
<30      1513
Name: count, dtype: int64
income_bracket
5k–10k     5929
2.5k–5k    5720
<2.5k      2442
10k+       2091
Name: count, dtype: int64
debt_bucket
Low        7282
Medium     5478
High       2134
Extreme     960
Name: count, dtype: int64
util_band
Low          6045
Maxed        3201
High         2557
Medium       2518
Overlimit    1012
Name: count, dtype: int64


### Feature Engineering Summary

New features were engineered to support grouped analysis and simplify SQL querying:

- **`total_late`**: Sum of late_30_59, late_60_89, and late_90 delinquencies
- **`any_late`**: Binary indicator if borrower had any delinquency
- **Binned Features**:
  - `age_group`: `<30`, `30–45`, `45–60`, `60+`
  - `income_bracket`: `<2.5k`, `2.5k–5k`, `5k–10k`, `10k+`
  - `debt_bucket`: `Low`, `Medium`, `High`, `Extreme`
  - `util_band`: `Low`, `Medium`, `High`, `Maxed`, `Overlimit`

These features will help evaluate credit risk trends across meaningful financial segments.


In [27]:
df.to_csv('../data/clean_credit_risk.csv', index=False)

### Next Steps

With a clean, structured dataset in place, the next step is to export it as a csv and use that with SQLite and begin writing SQL queries to explore patterns in borrower delinquency across income levels, credit utilization bands, and debt-to-income ratios.
